In [49]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch import nn,optim
from mpl_toolkits.mplot3d import Axes3D
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch.nn.functional as F
from sklearn.linear_model import LogisticRegression

In [17]:
torch.manual_seed(1)

In [24]:
cd C:\Users\Marcus\Desktop\python\alf_sports_betting_predictor

C:\Users\Marcus\Desktop\python\alf_sports_betting_predictor


In [25]:
df = pd.read_csv('2021_2020_afl_season_and_stats.csv')

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207 entries, 0 to 206
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               207 non-null    int64  
 1   Home_Team           207 non-null    object 
 2   Home_Team_Score     207 non-null    int64  
 3   Away_Team           207 non-null    object 
 4   Away_Team_Score     207 non-null    int64  
 5   home_Team           207 non-null    object 
 6   away_Team           207 non-null    object 
 7   home_Kicks          207 non-null    float64
 8   away_Kicks          207 non-null    float64
 9   home_Handballs      207 non-null    float64
 10  away_Handballs      207 non-null    float64
 11  home_Disposals      207 non-null    float64
 12  away_Disposals      207 non-null    float64
 13  home_Marks          207 non-null    float64
 14  away_Marks          207 non-null    float64
 15  home_Hit_outs       207 non-null    float64
 16  away_Hit

In [27]:
df

,index,Home_Team,Home_Team_Score,Away_Team,Away_Team_Score,home_Team,away_Team,home_Kicks,away_Kicks,home_Handballs,...,home_Frees_Against,away_Frees_Against,home_Goals,away_Goals,home_Behinds,away_Behinds,home_AFL_Fantasy,away_AFL_Fantasy,home_win_lose,away_win_lose
0,0,Richmond,105,Carlton,81,Richmond,Carlton,0.290415,0.775393,0.541276,...,1.000000,0.515789,0.811111,0.666667,0.786667,0.773333,0.244397,0.459011,1.0,0.0
1,1,Western Bulldogs,34,Collingwood,86,Western Bulldogs,Collingwood,0.434907,0.402003,1.000000,...,0.000000,0.200000,1.000000,0.500000,1.000000,0.120000,0.798588,0.529936,0.0,1.0
2,2,Essendon,63,Fremantle,57,Essendon,Fremantle,0.296137,0.380544,0.716698,...,0.621053,0.305263,0.744444,0.488889,0.360000,0.680000,0.553270,0.347866,1.0,0.0
3,3,Adelaide Crows,71,Sydney Swans,74,Adelaide Crows,Sydney Swans,0.000000,0.587983,0.497186,...,0.589474,0.600000,0.544444,0.800000,0.560000,0.386667,0.000000,0.481732,0.0,1.0
4,4,GWS Giants,105,Geelong Cats,73,GWS Giants,Geelong Cats,0.527897,0.782546,0.518762,...,0.589474,0.715789,0.566667,0.955556,0.520000,0.666667,0.402825,1.000000,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,49,Melbourne,82,Richmond,48,Melbourne,Richmond,0.668097,0.290415,0.590994,...,0.242105,1.000000,0.844444,0.811111,0.826667,0.786667,0.832668,0.244397,1.0,0.0
203,50,Fremantle,99,North Melbourne,48,Fremantle,North Melbourne,0.380544,0.203147,0.542214,...,0.305263,0.642105,0.488889,0.000000,0.680000,0.053333,0.347866,0.193430,1.0,0.0
204,51,Hawthorn,102,Adelaide Crows,99,Hawthorn,Adelaide Crows,0.248927,0.000000,0.744841,...,0.852632,0.589474,0.544444,0.544444,0.000000,0.560000,0.342340,0.000000,0.0,1.0
205,52,Collingwood,85,Essendon,109,Collingwood,Essendon,0.402003,0.296137,0.633208,...,0.200000,0.621053,0.500000,0.744444,0.120000,0.360000,0.529936,0.553270,1.0,0.0


In [89]:
df['home_adv'] = .75
df['away_disadv'] = .25

In [90]:
df.corr()[['home_win_lose', 'away_win_lose']]

,home_win_lose,away_win_lose
index,-0.016326,0.021425
Home_Team_Score,0.345952,-0.334493
Away_Team_Score,-0.347248,0.355638
home_Kicks,0.176941,-0.189698
away_Kicks,-0.124116,0.141752
home_Handballs,-0.080285,0.086252
away_Handballs,-0.010402,-0.000128
home_Disposals,0.043068,-0.045975
away_Disposals,-0.104035,0.105083
home_Marks,0.184547,-0.197951


In [91]:
#df = remove_zeroes_from_dataset(df, 'bathrooms')

In [92]:
x = df.drop(columns=['away_win_lose', 'home_win_lose', 'Home_Team_Score', 'Away_Team_Score', 'index', 
                    'Home_Team', 'Away_Team', 'home_Team', 'away_Team'])
y = df['home_win_lose']

In [93]:
x['home_adv'] = .75
x['away_disadv'] = .25

In [96]:
x

,home_Kicks,away_Kicks,home_Handballs,away_Handballs,home_Disposals,away_Disposals,home_Marks,away_Marks,home_Hit_outs,away_Hit_outs,...,home_Frees_Against,away_Frees_Against,home_Goals,away_Goals,home_Behinds,away_Behinds,home_AFL_Fantasy,away_AFL_Fantasy,home_adv,away_disadv
0,0.290415,0.775393,0.541276,0.168856,0.341491,0.278087,0.217304,0.444668,0.127841,0.457386,...,1.000000,0.515789,0.811111,0.666667,0.786667,0.773333,0.244397,0.459011,0.75,0.25
1,0.434907,0.402003,1.000000,0.633208,1.000000,0.536151,0.344064,0.360161,0.000000,1.000000,...,0.000000,0.200000,1.000000,0.500000,1.000000,0.120000,0.798588,0.529936,0.75,0.25
2,0.296137,0.380544,0.716698,0.542214,0.553949,0.412681,0.219316,0.408451,0.139205,0.267045,...,0.621053,0.305263,0.744444,0.488889,0.360000,0.680000,0.553270,0.347866,0.75,0.25
3,0.000000,0.587983,0.497186,0.364916,0.063404,0.362625,0.000000,0.285714,0.463068,0.477273,...,0.589474,0.600000,0.544444,0.800000,0.560000,0.386667,0.000000,0.481732,0.75,0.25
4,0.527897,0.782546,0.518762,0.637899,0.500556,0.838710,0.321932,0.845070,0.116477,0.210227,...,0.589474,0.715789,0.566667,0.955556,0.520000,0.666667,0.402825,1.000000,0.75,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,0.668097,0.290415,0.590994,0.541276,0.694105,0.341491,0.511066,0.217304,0.860795,0.127841,...,0.242105,1.000000,0.844444,0.811111,0.826667,0.786667,0.832668,0.244397,0.75,0.25
203,0.380544,0.203147,0.542214,0.552533,0.412681,0.286986,0.408451,0.303823,0.267045,0.500000,...,0.305263,0.642105,0.488889,0.000000,0.680000,0.053333,0.347866,0.193430,0.75,0.25
204,0.248927,0.000000,0.744841,0.497186,0.551724,0.063404,0.434608,0.000000,0.247159,0.463068,...,0.852632,0.589474,0.544444,0.544444,0.000000,0.560000,0.342340,0.000000,0.75,0.25
205,0.402003,0.296137,0.633208,0.716698,0.536151,0.553949,0.360161,0.219316,1.000000,0.139205,...,0.200000,0.621053,0.500000,0.744444,0.120000,0.360000,0.529936,0.553270,0.75,0.25


In [97]:
train_x, test_x, train_y, test_y =  train_test_split(x, y, test_size=.2)

In [98]:
log_model = LogisticRegression()

In [110]:
log_model.fit(train_x, train_y)

LogisticRegression()

In [111]:
y_pred = log_model.predict(test_x)
metrics.accuracy_score(test_y, y_pred)

0.5714285714285714

In [112]:
y_pred_train = log_model.predict(train_x)
metrics.accuracy_score(train_y, y_pred_train)

0.6484848484848484

In [114]:
from sklearn.model_selection import cross_val_score

In [115]:
log_model = LogisticRegression()

In [116]:
score = cross_val_score(log_model, x, y, cv=10)

In [117]:
score

array([0.52380952, 0.66666667, 0.80952381, 0.52380952, 0.71428571,
       0.57142857, 0.76190476, 0.65      , 0.4       , 0.35      ])

In [118]:
sum(score) / len(score)

0.5971428571428572